# Playbook title

## Introduction

- UUID: **UUID**
- State: **STATE**
- Purpose: Purpose
- - Purpose details
- Tags: [ "tags" ]
- External resources: **Mattermost, TheHive**
- Target audience: **SOC, CSIRT**
- Graphical workflow 


# Playbook

- **Playbook title**
- - Introduction
- **Preparation**
- - PR:1 Initialise environment
- - PR:2 Load helper functions
- - PR:3 Load helper variables
- - PR:4 MISP event details
- - PR:5 Setup MISP event link
- **Summary**
- - EN:xx Send a summary to Mattermost
- - EN:xx Send an alert to TheHive
- - EN:xx End of the playbook
- External references
- Technical details

# Preparation

## PR:1 Initialise environment

This section **initialises the playbook environment** and loads the required Python libraries. 

The credentials for MISP (**API key**) and other services are loaded from the file `keys.py` in the directory **vault**. A [PyMISP](https://github.com/MISP/PyMISP) object is created to interact with MISP and the active MISP server is displayed. By printing out the server name you know that it's possible to connect to MISP. In case of a problem PyMISP will indicate the error with `PyMISPError: Unable to connect to MISP`.

The contents of the `keys.py` file should contain at least :

```
misp_url="<MISP URL>"                  # The URL to our MISP server
misp_key="<MISP API KEY>"              # The MISP API key
misp_verifycert=<True or False>        # Indicate if PyMISP should attempt to verify the certificate or ignore errors
```

In [ ]:
# Initialise Python environment
import urllib3
import sys
import json
from pyfaup.faup import Faup
from prettytable import PrettyTable, MARKDOWN
from IPython.display import Image, display, display_markdown, HTML
from datetime import date
import requests
import uuid
from uuid import uuid4
from pymisp import *
from pymisp.tools import GenericObjectGenerator

# Load the credentials
sys.path.insert(0, "../vault/")
from keys import *
if misp_verifycert is False:
    import urllib3
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
print("The \033[92mPython libraries\033[90m are loaded and the \033[92mcredentials\033[90m are read from the keys file.")

# Create the PyMISP object
misp = PyMISP(misp_url, misp_key, misp_verifycert)
print("I will use the MISP server \033[92m{}\033[90m for this playbook.\n\n".format(misp_url))

## PR:2 Load helper functions

The next cell contains **helper functions** that are used in this playbook. 

Instead of distributing helper functions as separate Python files this playbook includes all the required code as one code cell. This makes portability of playbooks between instances easier. The downside is that functions defined in this playbook need to be defined again in other playbooks, which is not optimal for code re-use. For this iteration of playbooks it is chosen to include the code in the playbook (more portability), but you can easily create one "helper" file that contains all the helper code and then import that file in each playbook (for example by adding to the previous cell `from helpers import *`). Note that the graphical workflow image is included as an external image. A missing image would not influence the further progress of the playbook.

To avoid cluttering the output of the playbook the next code cell is **collapsed**. You still **need to execute it**, but you can leave the cell collapsed. Click on the cell to expand it, click on the left bar that indicates the active cell in the Jupyter notebook to collapse it again.

In [ ]:
def pb_get_misp_tags(tags=[], local_tags=[]):
    '''
    Get a list of MISP tags based on a Python list

    :param misp: MISP object
    :param object_template: which object template to return
    '''
    misp_tags = []
    for el in tags:
        t = MISPTag()
        t.name = el
        t.local = False
        misp_tags.append(t)

    for el in local_tags:
        t = MISPTag()
        t.name = el
        t.local = True
        misp_tags.append(t)
    return misp_tags

print("\033[92mHelper functions loaded\033[90m.\n\n".format(misp_url))

## PR:3 Set helper variables

This cell contains **helper variables** that are used in this playbook. Their usage is explained in the next steps of the playbook.

- `playbook_objects` : a dictionary of the objects that are created when the playbook progresses

## PR:4 MISP event details

### Event title

Now decide if you want to create a **new** MISP event or add data to an **existing** MISP event (referenced via its UUID). If you create a new event then its good practice to choose a self-explanatory **event title**. This event title is shown in the MISP event index and should provide you the necessary information what the event is about. You should avoid using generic event titles. Read the [Best Practices in Threat Intelligence](https://www.misp-project.org/best-practices-in-threat-intelligence.html) for further guidance.

### Contexualisation

This playbook adds event contexualisation via the **tags** that are defined in `event_additional_global_tags` (for *global* tags) and `event_additional_global_tags` (for *local* tags). As a reminder, whereas *global* tags remain attached to the events that you share with your community, the *local* tags are not shared outside your organisation. It's also a good idea to primarily use tags that are part of a [taxonomy](https://github.com/MISP/misp-taxonomies), this allows you to make the contexualisation more portable accross multiple MISP instances.

In this playbook the list of tags is build via one of the helper functions `pb_get_misp_tags`. This function takes two arguments, first a list of tags to convert as *global* tags, and secondly a list of tags to convert as *local* tags. 

### Traffic Light Protocol

The default **TLP** for this event is **<span style='color:#FFBF00'>tlp:amber</span>**. The Traffic Light Protocol (TLP) facilitates sharing of potentially sensitive information and allows for more effective collaboration. TLP is a set of four standard labels to indicate the sharing boundaries to be applied by the recipients. TLP is always set by the creator of information. You can find more information at [FIRST](https://www.first.org/tlp/). You can specify TLP via `event_tlp`.

### MISP distribution, threat level and analysis level

Optionally you can specifiy a MISP **distribution** (with `event_distribution`), **threat level** (with `event_threat_level_id`) or **analysis state** (with `event_analysis`). The event **date** is set to today via `event_date`.

If you cannot remember the options for distribution, threat level or the analysis state then use the next cell to guide you. This cell is set as **raw**. If you **change its type to code** and execute the cell you get an overview of the options available for creating a MISP event.

## PR:5 Setup MISP event link

Provide your **user input** via the variable `event_input`.

If you want to **create a new event** then provide the MISP event title in `event_input`. We will use the PyMISP function [add_event](https://pymisp.readthedocs.io/en/latest/_modules/pymisp/api.html#PyMISP.add_event) to create the MISP event.

If you want to add the results of the playbook to an **existing event**, then add the MISP event UUID to `event_input`. In this case we check that you provide a valid UUID4 and use the PyMISP function [get_event](https://pymisp.readthedocs.io/en/latest/_modules/pymisp/api.html#PyMISP.get_event) to query the MISP server for the provided UUID. We also request the response to be returned as a PyMISP object with `pythonify=True`. If the event does not exist then PyMISP will return an error.

In [ ]:
# Provide the event title for a new event or valid MISP event UUID to add to an existing event
#event_input = "9cf8cac1-845e-44db-af1f-266e858d63b5"
event_input = "Demo Event Create MISP objects and relations"

# Optionally, you can change TLP, add additional event (local and global) tags, threatlevel, analysis state or distribution level
event_tlp = "tlp:amber"

# Event context
event_additional_global_tags = []                                 # This needs to be a Python list
event_additional_local_tags = ["workflow:state=\"incomplete\""]   # This needs to be a Python list

# Additional MISP event settings
event_threat_level_id = ThreatLevel.low
event_analysis = Analysis.ongoing
event_distribution = Distribution.your_organisation_only
event_date = date.today()

In [ ]:
# Code block to create the event or add data to an existing event
event_input = event_input.strip()
new_event = False
misp_event = False

# Check if we received a valid UUID
try:
    uuid.UUID(str(event_input))
except ValueError:
    new_event = True

# Decide if we start with a new event or add to an existing one
if not new_event:   
    misp_event = misp.get_event(event_input, pythonify=True)
    if "errors" in misp_event:
        misp_event = False
        print("\033[91mUnable to proceed.\033[90m \nEither the MISP event does not exist or the UUID is not in the correct format.")
        print("Go back to the previous (code) cell and adjust the input.\n\n")
    else:
        print("Continue the playbook with adding data to an \033[92mexisting MISP event\033[90m: {} ({}).\n\n".format(misp_event.info, misp_event.uuid))
else:
    # Set the tags, or default to an empty list in case of wrong types
    if type(event_tlp) == str and type(event_additional_global_tags) == list and type(event_additional_local_tags) == list:
        event_additional_global_tags.append(event_tlp)
        misp_event_tags = pb_get_misp_tags(event_additional_global_tags, event_additional_local_tags)
    else:
        print("\033[91mWrong types\033[90m specified for the tags. I will not add tags to the event.")
        misp_event_tags = []
        
    # Create the PyMISP object for an event
    event = MISPEvent()
    event.info = event_input
    event.distribution = event_distribution
    event.threat_level_id = event_threat_level_id
    event.analysis = event_analysis
    event.set_date(event_date)
 
    # Create the MISP event on the server side
    misp_event = misp.add_event(event, pythonify=True)
    print("Continue the playbook with the new \033[92mcreated\033[90m MISP event ID {} with UUID {}".format(misp_event.id, misp_event.uuid))
    for el in misp_event_tags:
        misp.tag(misp_event.uuid, el, local=el.local)
        print("\033[92mAdded\033[90m event tag {}".format(el.name))
    print("\n\n")

# Investigate

# Closure

In this **closure** or end step we create a **summary** of the actions that were performed by the playbook. The summary is printed in the playbook and can also be send to a chat channel. 

In [ ]:
intro = "Demo"
summary = "Demo"
indicator_search = []

## EN:xx Send a summary to Mattermost

Now you can send the summary to Mattermost. You can send the summary in two ways by selecting one of the options for the variable `send_to_mattermost_option` in the next cell.

- The default option where the entire summary is in the **chat**, or
- a short intro and the summary in a **card**

For this playbook we rely on a webhook in Mattermost. You can add a webhook by choosing the gear icon in Mattermost, then choose Integrations and then **Incoming Webhooks**. Set a channel for the webhook and lock the webhook to this channel with *"Lock to this channel"*.

In [ ]:
send_to_mattermost_option = "via a chat message"
#send_to_mattermost_option = "via a chat message with card"

In [ ]:
message = False
if send_to_mattermost_option == "via a chat message":
    message = {"username": mattermost_playbook_user, "text": summary}
elif send_to_mattermost_option == "via a chat message with card":
    message = {"username": mattermost_playbook_user, "text": intro, "props": {"card": summary}}

if message:
    r = requests.post(mattermost_hook, data=json.dumps(message))
    r.raise_for_status()
if message and r.status_code == 200:
    print("Summary is \033[92msent to Mattermost.\n")
else:
    print("\033[91mFailed to sent summary\033[90m to Mattermost.\n")

## EN:xx Send an alert to TheHive

Next to informing your colleagues via Mattermost you can also send an **alert** to TheHive. The alert contains the summary, and a list of indicators as 'observables'.

You can change the alert title with `thehive_alert_title` and provide a reference type with `thehive_alert_reference`. Note that this reference needs to be **unique** in TheHive. If you want to create multiple alerts for the same MISP event then add some random value at the end.

In [ ]:
# The title of the TheHive alert
thehive_alert_title = "MISP Playbook Summary"

# A unique reference for the TheHive (we include the MISP event UUID)
thehive_alert_reference = "MISP event - {} - {}".format(misp_event.info, misp_event.uuid)

# Alert type in TheHive
thehive_alert_type = "MISP Playbook alert"

# TLP:Amber for TheHive
thehive_tlp = 2

# PAP:GREEN for TheHive
thehive_pap = 1

In [ ]:
# Code block to send an alert to TheHive
# We use the Python requests library
thehive_headers = {'Content-Type': 'application/json', 'Authorization': f'Bearer {thehive_key}'}
thehive_url_create = "{}/api/v1/alert".format(thehive_url)

thehive_observables = []
for indicator in indicator_search:
    dataType = False
    if indicator.type == "ip-src" or indicator.type == "ip-dst":
        dataType = "ip"
    elif indicator.type == "url":
        dataType = "url"
    elif indicator.type == "hostname":
        dataType = "hostname"        
    elif indicator.type == "email-dst" or indicator.type == "email-src":        
        dataType = "mail"
    if dataType:
        thehive_observables.append({"dataType": dataType, "data": indicator.value, "pap": thehive_pap, "tlp": thehive_tlp})
thehive_alert = {"title": thehive_alert_title, 
                 "description": intro,
                 "summary": summary[0:1048576],
                 "type": thehive_alert_type, 
                 "source": "playbook", 
                 "sourceRef": thehive_alert_reference, 
                 "tlp": thehive_tlp, "pap": thehive_pap,
                 "observables": thehive_observables}

result = requests.post(thehive_url_create, headers=thehive_headers, data=json.dumps(thehive_alert))
if result.json()['status'] == 'New':
    thehive_alert_id = result.json()['_id']
    print('The TheHive \033[92malert {} is added'.format(thehive_alert_id))
else:
    print('\033[91mFailed\033[90m to add TheHive alert')
    print(result.text)

## EN:xx End of the playbook 

In [ ]:
print("\033[92m End of the playbook")


## External references <a name="extreferences"></a>

- [The MISP Project](https://www.misp-project.org/)
- [Mattermost](https://mattermost.com/)
- [TheHive](https://thehive-project.org/)

## Technical details 

### Documentation

This playbook requires these Python **libraries** to exist in the environment where the playbook is executed. You can install them with `pip install <library>`.

```
pyfaup
chardet
PrettyTable
ipywidgets
```

You need to have **network access** to 
- your MISP server (HTTP or HTTPS)

You need
- an **API key with MISP**
- - Under Global Actions, My Profile. Add an extra authentication key.
- - Add the API key (`misp_key`) and the MISP URL (`misp_url`) to `keys.py`
- - If you use a self-signed certificate set `misp_verifycert` to False

### Helper functions

### Colour codes

The output from Python displays some text in different colours. These are the colour codes

```
Red = '\033[91m'
Green = '\033[92m'
Blue = '\033[94m'
Cyan = '\033[96m'
White = '\033[97m'
Yellow = '\033[93m'
Magenta = '\033[95m'
Grey = '\033[90m'
Black = '\033[90m'
Default = '\033[99m'
```